Essaye de créer un arbre des catégories en partant des catégories considérés comme les plus hautes par Wikipédia (https://fr.wikipedia.org/wiki/Portail:Accueil)

In [58]:
import csv, os, bz2, time, re, json, gzip

from collections import Counter
from coproximity_create_vocabulary.data_conf import base_vocab_folder


In [2]:
wiki_tilte_fr_folder = base_vocab_folder + 'whole/vocabulary/wiki_title_fr/'
dump_folder = wiki_tilte_fr_folder + 'dumps/'
page_file = dump_folder + 'frwiki-latest-page.sql.gz'
redirect_file = dump_folder + 'frwiki-latest-redirect.sql.gz'

category_folder = base_vocab_folder + 'whole/vocabulary/wiki_title_fr/categories/'
categorylinks_file = category_folder + 'frwiki-20220601-categorylinks.sql.gz'
categorylinks_json_file = category_folder + 'categories.json'

In [3]:
id2title = {}
names_space = set()
with gzip.open(page_file)as f  :
    for line in f :
        if line.startswith(b'INSERT INTO `page` VALUES') :
            split_insert = [x.strip('(') for x in line.decode('utf8').strip(');\n)').split('VALUES ')[1].split('),(')]

            for vals in csv.reader(split_insert, delimiter=',', quotechar="'", escapechar ='\\') :
                #Only get the id of Wikipedia pages which namespace are "Main" 
                if int(vals[1]) == 14 and not ':' in vals[2] :
                    id2title[vals[0]] = vals[2]

In [4]:
title2id = {title: id_ for id_ , title in id2title.items() }
cat_id = set(id2title)
cat_title = set(title2id)

In [5]:
id2title

{'80753': 'Astronomie',
 '80787': 'Écologie',
 '81060': 'Sept_Merveilles_du_monde',
 '81355': 'Environnement',
 '81357': 'Biologie',
 '81359': 'Agriculture',
 '81422': 'Système_solaire',
 '81442': 'Architecture',
 '81455': 'Pluton',
 '81473': 'Soleil',
 '81482': 'Vénus',
 '81488': 'Terre',
 '81490': 'Mars',
 '81538': 'Physique',
 '81540': 'Chimie',
 '81541': 'Science',
 '81542': 'Mathématiques',
 '81545': 'Géographie',
 '81546': 'Zoologie',
 '81547': 'Botanique',
 '81630': 'Économie',
 '81646': 'Économie_politique',
 '81648': 'Finance',
 '81651': 'Monde_indien',
 '81894': 'Seconde_Guerre_mondiale',
 '81895': 'Phonétique',
 '81896': 'Linguistique',
 '81956': 'Mondialisation',
 '81979': 'Punk',
 '81980': 'Protéine',
 '81983': 'Ska',
 '82073': 'Génétique',
 '82094': 'Sciences_de_la_Terre',
 '82103': 'Chimiste',
 '82142': 'Constellation',
 '82159': 'Acteur_français',
 '82187': 'Amérique_centrale',
 '82192': 'Amérique_du_Sud',
 '82194': 'Amérique',
 '82199': 'Afrique',
 '82204': 'Mythologie

In [6]:
cat_id

{'7314561',
 '8118135',
 '1541173',
 '13390641',
 '6619560',
 '13514434',
 '2897154',
 '7004294',
 '13688941',
 '7351095',
 '13245381',
 '7252966',
 '4681585',
 '9822752',
 '11826725',
 '7938265',
 '9421585',
 '11815709',
 '12599380',
 '5532719',
 '12933327',
 '14084171',
 '7216416',
 '1181031',
 '7952349',
 '10440616',
 '3733556',
 '7355888',
 '14419761',
 '11351296',
 '1066384',
 '12464254',
 '10988278',
 '14872362',
 '9573252',
 '10254407',
 '3161177',
 '5867631',
 '7471273',
 '12002650',
 '11648324',
 '14819937',
 '13051053',
 '10495222',
 '7332497',
 '6684471',
 '1138074',
 '753655',
 '993169',
 '7960926',
 '12890036',
 '6537346',
 '3208759',
 '9364272',
 '10108735',
 '6138680',
 '5303273',
 '7930496',
 '3178062',
 '8725906',
 '13441576',
 '6619166',
 '6321300',
 '5913288',
 '14374059',
 '8410011',
 '6403349',
 '4430721',
 '9447717',
 '13900648',
 '12036207',
 '10860080',
 '1663212',
 '7285605',
 '6279621',
 '10878285',
 '12367736',
 '13984573',
 '14864590',
 '783396',
 '13078644'

In [8]:
if os.path.exists(categorylinks_json_file) :
    with open(categorylinks_json_file, encoding='utf8') as f :
        subcats = json.load(f)
else :
    subcats = {title: [] for title in cat_title}
    with gzip.open(categorylinks_file)as f  :
        for i, line in enumerate(f) :
            if line.startswith(b'INSERT INTO `categorylinks` VALUES') :
                split_insert = [x.strip('(') for x in line.decode('utf8', errors='ignore').strip(');\n)').split('VALUES ')[1].split('),(')]

                for vals in csv.reader(split_insert, delimiter=',', quotechar="'", escapechar ='\\') :
                    #Only get the id of Wikipedia pages which namespace are "Main" 
                    if vals[0] in cat_id and vals[1] in cat_title and not ':' in vals[1] :
                        subcats[vals[1]].append(id2title[vals[0]])

    with open(categorylinks_json_file, 'w', encoding='utf8') as f :
        json.dump(subcats, f)

In [9]:
start_categories = [
    'Science', 'Technologie', 'Médecine', 'Art', 'Sport', 'Loisir', 'Société', 'Politique', 
    'Croyance', 'Histoire', 'Géographie',
]
len(start_categories)

11

In [10]:
cat_tree = {title:[] for title in cat_title}
#assigner à cat_tree tous les paths qui mennent vers les catégories

In [56]:
cat_tree = {title:[] for title in cat_title}
current_tree = [ (cat, '') for cat in start_categories]
i = 0
while current_tree :
    if not i % 100000 :
        print(i, len(current_tree), len(cat_tree), current_tree[0])
    curr_cat, curr_tree_path = current_tree.pop(0)
    if not cat_tree[curr_cat] :
        current_tree.extend([(cat, f'{curr_tree_path}{curr_cat}/') for cat in subcats[curr_cat]])
    cat_tree[curr_cat].append(curr_tree_path)
    i += 1

0 11 522653 ('Science', '')
100000 238519 522653 ('Athlète_(femme)_slovène', 'Sport/Sport_par_pays/Sport_en_Slovénie/Athlétisme_en_Slovénie/')
200000 310939 522653 ("Ministre_malien_de_l'Environnement_et_de_l'Assainissement", "Politique/Politique_par_domaine_d'intervention/Politique_de_l'environnement/Ministre_de_l'Environnement_ou_équivalent/")
300000 360011 522653 ('Football_en_1988', 'Sport/Chronologie_du_sport/Chronologie_du_football/Football_au_XXe_siècle/Football_dans_les_années_1980/')
400000 318236 522653 ('Productrice_française_de_télévision', 'Société/Médias/Personnalité_du_monde_des_médias/Productrice/Productrice_de_télévision/')
500000 321412 522653 ('Député_européen_élu_en_Tchéquie_2014-2019', 'Politique/Personnalité_politique/Personnalité_politique_par_nationalité/Personnalité_politique_tchèque/Député_européen_élu_en_Tchéquie/')
600000 341941 522653 ('Logiciel_écrit_en_C_et_assembleur', 'Science/Domaine_scientifique/Techniques_et_sciences_appliquées/Informatique/Logiciel/

In [73]:
min_path_cat_tree = {
    cat: [path for path in tree if path.count('/') == min(y.count('/') for y in tree)]
    for cat, tree in cat_tree.items() if tree
}

In [88]:
len([(cat, tree) for cat, tree in min_path_cat_tree.items() if len(tree) > 2]), len(min_path_cat_tree),\
    [(cat, tree) for cat, tree in min_path_cat_tree.items() if len(tree) > 2]

(32780,
 490804,
 [("Historien_de_l'art_néerlandais",
   ["Art/Histoire_de_l'art/Historien_de_l'art/Historien_de_l'art_par_nationalité/",
    "Art/Histoire_de_l'art/Histoire_de_l'art_par_pays/Histoire_de_l'art_aux_Pays-Bas/",
    'Histoire/Historien/Historien_par_nationalité/Historien_néerlandais/']),
  ('Tirreno-Adriatico',
   ['Sport/Sport_par_pays/Sport_en_Italie/Événement_sportif_en_Italie/Événement_sportif_international_en_Italie/',
    'Sport/Catégorie_par_sport/Compétition_par_sport/Compétition_cycliste/Calendrier_mondial_UCI/',
    'Sport/Catégorie_par_sport/Compétition_par_sport/Compétition_cycliste/UCI_World_Tour/',
    'Société/Compétition/Compétition_par_date_de_fondation/Compétition_fondée_au_XXe_siècle/Compétition_fondée_en_1966/']),
  ('Naissance_dans_le_Territoire_sous_tutelle_des_îles_du_Pacifique',
   ['Société/Personne/Personnalité_par_lieu_de_naissance/Naissance_par_continent/Naissance_en_Océanie/',
    'Société/Personne/Personnalité_par_lieu_de_naissance/Naissance_

In [75]:
list(sorted(
   min_path_cat_tree.items(),
    key = lambda x : min((y.count('/'), len(y)) for y in x[1]) 
))[1000:2000]

[('Sécurité_des_transports', ['Société/Sécurité/']),
 ('Diable', ['Société/Religion/', 'Croyance/Légende/']),
 ('Forme_de_handicap', ['Société/Handicap/']),
 ('Personnalité_par_genre', ['Société/Personne/']),
 ('Académie_de_musique_à_la_Renaissance', ['Société/Académie/']),
 ('Académies_royales_de_Suède', ['Société/Académie/']),
 ('Naturisme_au_cinéma',
  ['Loisir/Naturisme/', 'Loisir/Film_sur_le_thème_des_vacances/']),
 ('Accessibilité', ['Société/Handicap/']),
 ('Personnalité_par_organisation',
  ['Société/Organisation/', 'Société/Personne/']),
 ("Dénomination_d'une_personne", ['Société/Personne/']),
 ('Académie_par_pays', ['Société/Académie/']),
 ('Catégorie_par_religion', ['Société/Religion/']),
 ('Charité',
  ['Société/Religion/', 'Société/Organisation_sociale/', 'Société/Pauvreté/']),
 ('Rom_et_religion', ['Société/Religion/']),
 ('Film_documentaire_sur_la_pauvreté', ['Société/Pauvreté/']),
 ('Biographie', ['Société/Personne/']),
 ('Banque_alimentaire', ['Société/Aide_humanitaire

In [49]:

curr_cat, subcats[curr_cat], [(cat, f'{curr_tree_path}{cat}/') for cat in subcats[curr_cat] if cat_tree[cat]]

('Croyance',
 ['Mythologie',
  'Religion',
  'Spiritualité',
  'Astrologie',
  'Pseudo-science',
  'Légende',
  'Mysticisme',
  'Superstition',
  'Divinité',
  'Méditation',
  'Agnosticisme',
  'Satanisme',
  'Magie',
  'Dieu',
  'Croyance_médiévale',
  'Critique_des_croyances',
  'Séjour_des_morts',
  'Arbre_et_croyance',
  'Volcanisme_dans_les_croyances',
  'Fin_du_monde',
  'Esclavage_dans_les_croyances',
  'Divination',
  'Croyance_en_Bretagne',
  'Réincarnation_dans_la_fiction',
  'Liste_en_rapport_avec_les_croyances',
  'Cornucopien',
  'Cornucopianisme',
  'Résurrection',
  'Demi-divinité',
  'Opinion_publique'],
 [])

In [81]:
lit_subcategories = {cat for cat , tree in min_path_cat_tree.items() if all('Littérature' in path.split('/') for path in tree)}
lit_subcategories

{'Alcools_(Apollinaire)',
 'Allié_de_Harry_Potter',
 'Alphabet_runique',
 "Ancienne_maison_d'édition_bretonne",
 'Anthologie_composée_par_Gardner_R._Dozois',
 'Anthologie_composée_par_George_R._R._Martin',
 'Anthologie_en_littérature',
 'Anthologie_en_littérature_de_fantasy',
 'Anthologie_en_littérature_de_science-fiction',
 'Anthologiste',
 'Auteur_de_la_collection_Aventures_et_Passions',
 'Auteur_de_la_collection_Un_livre_dont_VOUS_êtes_le_héros',
 "Auteur_de_roman_d'amour",
 "Auteur_ou_Illustrateur_de_la_collection_C'est_toi",
 'Bibliobus',
 'Bibliothèque_Bodléienne',
 'Bibliothèque_Carnegie',
 'Bibliothèque_de_Québec',
 "Bibliothèque_de_l'Université_McGill",
 "Bibliothèque_de_l'Université_du_Texas",
 "Bibliothèque_de_l'université_d'Oxford",
 'Bibliothèque_du_Congrès',
 'Bibliothèque_du_Patrimoine_de_Clermont_Auvergne_Métropole',
 'Bibliothèque_municipale_de_Lyon',
 'Bibliothèque_municipale_de_Nantes',
 'Bibliothèque_nationale',
 "Bibliothèque_nationale_d'Algérie",
 'Bibliothèque_na

In [87]:
lit_subcategories = {cat for cat , tree in min_path_cat_tree.items() if all('Littérature' in path for path in tree)}
list(sorted(lit_subcategories))

['87e_District',
 "Adaptation_d'un_roman_angolais_au_cinéma",
 "Adaptation_d'un_roman_colombien_au_cinéma",
 "Adaptation_d'un_roman_croate_au_cinéma",
 "Adaptation_d'un_roman_géorgien_au_cinéma",
 "Adaptation_d'un_roman_israélien_au_cinéma",
 "Adaptation_d'un_roman_letton_au_cinéma",
 "Adaptation_d'un_roman_malaisien_au_cinéma",
 "Adaptation_d'un_roman_nigérian_au_cinéma",
 "Adaptation_d'un_roman_néo-zélandais_au_cinéma",
 "Adaptation_d'un_roman_pakistanais_au_cinéma",
 "Adaptation_d'un_roman_philippin_au_cinéma",
 "Adaptation_d'un_roman_somalien_au_cinéma",
 "Adaptation_d'un_roman_vénézuélien_au_cinéma",
 "Adaptation_d'une_nouvelle_croate_au_cinéma",
 "Adaptation_d'une_nouvelle_cubaine_au_cinéma",
 "Adaptation_d'une_nouvelle_islandaise_au_cinéma",
 "Adaptation_d'une_nouvelle_vietnamienne_au_cinéma",
 "Adaptation_d'une_œuvre_de_Jules_Verne",
 'Adaptation_d’une_œuvre_de_Rudyard_Kipling',
 'Alain-Fournier',
 'Alain_Georges_Leduc',
 'Alain_Soral',
 'Alcools_(Apollinaire)',
 'Alexandre_Sol

In [85]:
min_path_cat_tree['Littérature_française'], cat_tree['Littérature_française']

(['Art/Art_par_pays/Littérature_par_pays/'],
 ['Art/Art_par_pays/Littérature_par_pays/',
  'Art/Art_par_continent/Littérature_par_continent/Littérature_européenne/',
  'Société/Culture/Culture_nationale/Culture_française/',
  "Art/Art_par_forme_d'expression/Littérature/Littérature_par_langue/Littérature_francophone/",
  'Société/Communication/Langue/Langue_véhiculaire/Langue_française/'])